In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers
!pip install datasets
!pip install wordcloud
!pip install plotly
!pip install nlp
!pip install huggingface_hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
#Importing data manipulation libraried
import pandas as pd
import numpy as np

# Importing libraries from Hugging Face and WordCloud(text visualisation)
from transformers import pipeline
from wordcloud import WordCloud

#Importing Data visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import os

from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

from datasets import load_metric

# Import necessary libraries and tools for text processing and natural language processing (NLP)
import nltk  # Natural Language Toolkit
from nltk.tokenize import word_tokenize  # Tokenization
from nltk.corpus import stopwords  # Stopwords
from nltk.stem import WordNetLemmatizer  # Word Lemmatization

import nlp  # Hugging Face's NLP library

import torch  # PyTorch, a deep learning framework
import torch.nn.functional as F  # Functions for PyTorch neural networks

In [6]:
# Disable W&B
os.environ["WANDB_DISABLED"] = "true"

In [7]:
# Load the training dataset from a CSV file located at the specified path.
train_path = "/content/drive/MyDrive/p5/p5 dataset/train_data.csv"

# Read the CSV file into a Pandas DataFrame for initial exploration.
df = pd.read_csv(train_path)
print(df.head())

                                                text  label  agreement  \
0  amp big homie meanboy mb mb mmr stegmanlife st...    0.0        1.0   
1  think devote career prove autism cause vaccine...    1.0        1.0   
2           whatcausesautism vaccine vaccinate child   -1.0        1.0   
3  mean immunize kid something wo secretly kill y...   -1.0        1.0   
4  thanks catch perform la nuit nyc show start je...    0.0        1.0   

                                          lemmatized  
0  ['amp', 'big', 'homie', 'meanboy', 'mb', 'mb',...  
1  ['think', 'devote', 'career', 'prove', 'autism...  
2  ['whatcausesautism', 'vaccine', 'vaccinate', '...  
3  ['mean', 'immunize', 'kid', 'something', 'wo',...  
4  ['thanks', 'catch', 'perform', 'la', 'nuit', '...  


Splitting the Dataset

In [8]:
# Split the train data => {train, eval}
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [9]:
train.head()

,text,label,agreement,lemmatized
1641,new hey love mmr manymenrecords youaintgotnoea...,0.0,1.000000,"['new', 'hey', 'love', 'mmr', 'manymenrecords'..."
3907,new extends exemption charitable immunity cert...,0.0,1.000000,"['new', 'extends', 'exemption', 'charitable', ..."
336,esp mercury free vaccine available,1.0,0.666667,"['esp', 'mercury', 'free', 'vaccine', 'availab..."
6861,life entertainment yotc mmr exotics,0.0,1.000000,"['life', 'entertainment', 'yotc', 'mmr', 'exot..."
720,baby luna sore vaccine poorpuppy,0.0,0.666667,"['baby', 'luna', 'sore', 'vaccine', 'poorpuppy']"


Examples of Positive Tweets, i.e tweets with Label 1:

In [10]:
train[train['label'] == 1]['text'].values[:5]

array(['esp mercury free vaccine available',
       'reagan school send flyer offering flu insurance cost insurance cost tn care say',
       'case measles idiot need vaccinate really',
       'aid n bcg vaccination could improve survival hiv expose baby',
       'california governor sign strict school vaccine legislation jerry brown wasted time tuesday signing'],
      dtype=object)

Examples of Positive Tweets, i.e tweets with Label 0:

In [11]:
train[train['label'] == 0]['text'].values[:5]

array(['new hey love mmr manymenrecords youaintgotnoeazyinyourserato trappfly comingsoon',
       'new extends exemption charitable immunity certain civil action person age',
       'life entertainment yotc mmr exotics',
       'baby luna sore vaccine poorpuppy',
       'vaccine change course u disease time'], dtype=object)

Examples of Positive Tweets, i.e tweets with Label -1:

In [12]:
train[train['label'] == -1]['text'].values[:5]

array(['shld put statement shameful tell fb follower worry',
       'vaccine cause brain swell fever seizure ultimately brain encephalitis read package insert',
       'hearthiswell vaccine r link autism ibd peds note mmr mo cdcwhistleblower breakabillion',
       'could pay enough vaccinate way notachance',
       'son autism bear allergy food need learn vaccine glyphosate interact'],
      dtype=object)

In [13]:
eval.head()

,text,label,agreement,lemmatized
5818,nervous baby get vaccine clinic southwest,1.0,0.666667,"['nervous', 'baby', 'get', 'vaccine', 'clinic'..."
7842,aid n malaria vaccine child hiv plasmodium fal...,0.0,0.666667,"['aid', 'n', 'malaria', 'vaccine', 'child', 'h..."
880,measles outbreak hit texas church preach vacci...,1.0,0.666667,"['measles', 'outbreak', 'hit', 'texas', 'churc..."
9072,thank mtg look forward support increase child ...,1.0,1.000000,"['thank', 'mtg', 'look', 'forward', 'support',..."
288,health district offer immunization uninsured c...,1.0,0.666667,"['health', 'district', 'offer', 'immunization'..."


In [14]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (8000, 4), eval is (2001, 4)


In [15]:
# Save splitted subsets
train.to_csv("/content/drive/MyDrive/p5/p5 dataset/train_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/p5/p5 dataset/eval_subset.csv", index=False)

In [16]:
from datasets import load_dataset
dataset = load_dataset('csv',
                        data_files={'train': '/content/drive/MyDrive/p5/p5 dataset/train_subset.csv',
                        'eval': '/content/drive/MyDrive/p5/p5 dataset/eval_subset.csv'}, encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

# Tokenization and Data Preparation
### Tokenize and prepare data for model input

In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['lemmatized'], padding='max_length', truncation=True, return_tensors='pt')


#def tokenize_data(example):
#return tokenizer(example['text'], padding='max_length')

# Change the tweets to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = [ 'label', 'text', 'agreement','lemmatized']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2001
    })
})

In [20]:
!pip install accelerate>=0.20.1 transformers

In [1]:
from transformers import TrainingArguments
# Configure the training parameters like `num_train_epochs`.

training_args = TrainingArguments(output_dir="FineTuned-DistilBert-Model", num_train_epochs=3, load_best_model_at_end=True, push_to_hub=True,
                                  evaluation_strategy= "steps", save_strategy= 'steps')

In [21]:
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
train_dataset = dataset['train'].shuffle(seed=10)
eval_dataset = dataset['eval'].shuffle(seed=10)

In [23]:
from transformers import Trainer
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [24]:
# Launch the learning process: training
trainer.train()

Step,Training Loss,Validation Loss
500,0.794000,0.732847
1000,0.747000,0.728385
1500,0.676100,0.700007
2000,0.644400,0.666893
2500,0.538500,0.698189
3000,0.521600,0.698916


TrainOutput(global_step=3000, training_loss=0.6536216837565104, metrics={'train_runtime': 1369.4185, 'train_samples_per_second': 17.526, 'train_steps_per_second': 2.191, 'total_flos': 3179274264576000.0, 'train_loss': 0.6536216837565104, 'epoch': 3.0})

In [25]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-25-c203b637589b>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [27]:
# Launch the final evaluation
trainer.evaluate()

{'eval_loss': 0.6668931841850281,
 'eval_accuracy': 0.7216391804097951,
 'eval_runtime': 33.8342,
 'eval_samples_per_second': 59.141,
 'eval_steps_per_second': 7.419}

In [28]:
trainer.push_to_hub("FineTuned-DistilBert-Model")

events.out.tfevents.1699365873.70b9788ccb29.3906.1:   0%|          | 0.00/346 [00:00<?, ?B/s]

'https://huggingface.co/Afia-manubea/FineTuned-DistilBert-Model/tree/main/'

In [29]:
tokenizer.push_to_hub("FineTuned-DistilBert-Model")

CommitInfo(commit_url='https://huggingface.co/Afia-manubea/FineTuned-DistilBert-Model/commit/868d22525e30c36a59a5e230f87f4c558e9f91de', commit_message='Upload tokenizer', commit_description='', oid='868d22525e30c36a59a5e230f87f4c558e9f91de', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
model.push_to_hub("FineTuned-DistilBert-Model")

CommitInfo(commit_url='https://huggingface.co/Afia-manubea/FineTuned-DistilBert-Model/commit/93bee32d573c8e7c0711cc89ba1756d733a202b4', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='93bee32d573c8e7c0711cc89ba1756d733a202b4', pr_url=None, pr_revision=None, pr_num=None)

This table summarizes the key metrics for Model 1 and Model 2, making it easier to compare their performance.

| Metric                 | Model 1            | Model 2            |
|------------------------|--------------------|--------------------|
| Training Loss          | 0.677              | 0.638              |
| Training Runtime (s)   | 2721               | 1361               |
| Training Samples/Sec   | 8.819              | 17.623             |
| Evaluation Loss        | 0.663              | 0.669              |
| Evaluation Accuracy    | 0.739              | 0.723              |
| Evaluation Runtime (s) | 68.238             | 33.543             |
| Evaluation Samples/Sec | 29.324             | 59.655             |

From this table we observe that

* Model 2 has a lower training loss  compared to Model 1. While, Model 1 has a longer training runtime  compared to Model 2.

*  Model 2 has a higher training samples per second compared to Model 1. This indicates that Model 2 processes more training samples per second.

* Model 2 has a slightly higher evaluation loss (0.669) compared to Model 1 (0.663). A lower evaluation loss is usually desirable, but the difference is relatively small in this case.

* Model 1 has a higher evaluation accuracy (0.7386) compared to Model 2 (0.7231). A higher accuracy is much desirable, indicating better performance on the evaluation dataset.